In [ ]:
import pandas as pd
import numpy as np


In [ ]:
data = pd.read_csv("/content/IMDB Dataset.csv")
# Dataset is now stored in a Pandas DataFrame

Exploratory Data Analysis

In [ ]:
#Describing the no of rows and columns
data.shape

(50000, 2)

In [ ]:
#Describing the top 5 values of the dataset
data.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [ ]:
#display column data types and missing values
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   review     50000 non-null  object
 1   sentiment  50000 non-null  object
dtypes: object(2)
memory usage: 781.4+ KB


In [ ]:
#displaying summary statistics
data.describe()

,review,sentiment
count,50000,50000
unique,49582,2
top,Loved today's show!!! It was a variety and not...,positive
freq,5,25000


In [ ]:
data["sentiment"].value_counts()

,count
sentiment,
positive,25000
negative,25000


In [ ]:
# one hot encoding or label encoder
# positive -> 1
# negative -> 0

data.replace({"sentiment": {"positive":1, "negative":0}}, inplace=True)


<ipython-input-17-ee3f21815d45>:5: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data.replace({"sentiment": {"positive":1, "negative":0}}, inplace=True)


In [ ]:
!pip install tensorflow scikit-learn


In [ ]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
train_data, test_data= train_test_split(data, test_size = 0.2, random_state=42)

In [ ]:
train_data.shape

(40000, 2)

In [ ]:
test_data.shape

(10000, 2)

In [ ]:
tokenizer= Tokenizer(num_words = 5000)
tokenizer.fit_on_texts(train_data["review"])

In [ ]:
X_train = pad_sequences(tokenizer.texts_to_sequences(train_data["review"]), maxlen=200)
X_test = pad_sequences(tokenizer.texts_to_sequences(test_data["review"]), maxlen=200)

In [ ]:
X_train

array([[1935,    1, 1200, ...,  205,  351, 3856],
       [   3, 1651,  595, ...,   89,  103,    9],
       [   0,    0,    0, ...,    2,  710,   62],
       ...,
       [   0,    0,    0, ..., 1641,    2,  603],
       [   0,    0,    0, ...,  245,  103,  125],
       [   0,    0,    0, ...,   70,   73, 2062]], dtype=int32)

In [ ]:
X_test

array([[   0,    0,    0, ...,  995,  719,  155],
       [  12,  162,   59, ...,  380,    7,    7],
       [   0,    0,    0, ...,   50, 1088,   96],
       ...,
       [   0,    0,    0, ...,  125,  200, 3241],
       [   0,    0,    0, ..., 1066,    1, 2305],
       [   0,    0,    0, ...,    1,  332,   27]], dtype=int32)

In [ ]:
Y_train = train_data["sentiment"]
Y_test = test_data["sentiment"]

In [ ]:
#LSTM MODEL BUILDING

In [ ]:
model = Sequential()
model.add(Embedding(input_dim=5000, output_dim=128, input_length=200))
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation="sigmoid"))

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [ ]:
model.summary

<bound method Model.summary of <Sequential name=sequential, built=False>>

In [ ]:
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])
model.fit(X_train,Y_train, epochs=5, batch_size = 64, validation_split=0.2)

Epoch 1/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 149s 272ms/step - accuracy: 0.7045 - loss: 0.5506 - val_accuracy: 0.8454 - val_loss: 0.3577
Epoch 2/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 190s 269ms/step - accuracy: 0.8360 - loss: 0.3801 - val_accuracy: 0.8630 - val_loss: 0.3303
Epoch 3/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 142s 270ms/step - accuracy: 0.8769 - loss: 0.3061 - val_accuracy: 0.8380 - val_loss: 0.3641
Epoch 4/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 145s 276ms/step - accuracy: 0.8867 - loss: 0.2796 - val_accuracy: 0.8562 - val_loss: 0.3472
Epoch 5/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 139s 271ms/step - accuracy: 0.9071 - loss: 0.2407 - val_accuracy: 0.8708 - val_loss: 0.3466


In [ ]:
model.save("model_imdb.h5")

In [ ]:
import joblib
joblib.dump(tokenizer, "tokenizer.pkl")

['tokenizer.pkl']

In [ ]:
loss, accuracy = model.evaluate(X_test, Y_test)

313/313 ━━━━━━━━━━━━━━━━━━━━ 29s 90ms/step - accuracy: 0.8766 - loss: 0.3304


In [ ]:
print(loss, accuracy)

0.32940930128097534 0.8762999773025513


Building a Predictive system

In [ ]:
def predictive_system(review):
  sequences= tokenizer.texts_to_sequences([review])
  padded_sequence= pad_sequences(sequences, maxlen=200)
  prediction = model.predict(padded_sequence)
  sentiment = "positive" if prediction[0][0] > 0.5 else "negative"
  return prediction

In [ ]:
predictive_system("Brilliantly executed with a heartwarming story. Highly recommend it!")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 89ms/step


array([[0.9884693]], dtype=float32)

In [ ]:
predictive_system("I couldn't sit through the entire film. It was boring and overly predictable.")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 153ms/step


array([[0.02086179]], dtype=float32)

In [ ]:
predictive_system("Great acting, but the story dragged in places. Still worth a watch for the performances.")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 90ms/step


array([[0.9937721]], dtype=float32)